### Importing all the libraries

In [119]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

### Initailising the variables
end_date_for_ind_variable - Indicates the datetime upto which independent variables are <br /> 
campaign_launched - The date on which campaign was launched <br /> 
end_date_tar_variable - Indicates the datetime upto which contributions were accepted <br /> 
campaign_id - Id for selected campaigns <br /> 


In [120]:
end_date_for_ind_variable = datetime.datetime(2019, 5, 18)
campaign_launched = datetime.datetime(2019, 6, 18)
end_date_tar_variable = datetime.datetime(2019, 12, 18)
campaign_id = 7244

# Reading Data

In [121]:
campaigns = pd.read_csv('./data/campaigns.csv')

donors = pd.read_csv('./data/donors.csv')

gifts = pd.read_csv('./data/gifts.csv')

campaigns_campaign_id = pd.read_csv('./data/selection campaign 7244.csv')

# GIFTS

### Checking data in gifts

In [122]:

print(gifts.isna().sum().sum())
for column in gifts.columns[0:]:
    print(column + " " + str(gifts[column].isna().sum()))

gifts.info()

15637
Unnamed: 0 0
donorID 0
campaignID 15637
amount 0
date 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216594 entries, 0 to 216593
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  216594 non-null  int64  
 1   donorID     216594 non-null  int64  
 2   campaignID  200957 non-null  float64
 3   amount      216594 non-null  object 
 4   date        216594 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 8.3+ MB


In [123]:
# Dropping any duplicate gifts
gifts.drop_duplicates()

,Unnamed: 0,donorID,campaignID,amount,date
0,0,100001,NaN,"12,39",2004-03-23
1,1,100001,1577.0,"12,39",2007-10-11
2,2,100001,1634.0,"6,69",2007-12-28
3,3,100001,1778.0,"24,79",2008-06-14
4,4,100001,1808.0,"7,44",2008-07-09
...,...,...,...,...,...
216589,216589,265584,6169.0,"300,0",2018-09-23
216590,216590,281770,6169.0,"30,0",2018-09-08
216591,216591,317880,6169.0,"157,5",2018-09-12
216592,216592,480980,6169.0,"36,0",2018-09-12


### Cleaning data in gifts

In [124]:
gifts['campaignID'] = gifts['campaignID'].fillna(0.0)
gifts['campaignID'] = gifts['campaignID'].astype(int)
gifts['amount'] = gifts['amount'].str.replace(",", ".").astype(float)
gifts['date'] = pd.to_datetime(gifts['date'])
gifts.rename(columns={'Unnamed: 0' : 'giftID'}, inplace=True)
gifts.head()

,giftID,donorID,campaignID,amount,date
0,0,100001,0,12.39,2004-03-23
1,1,100001,1577,12.39,2007-10-11
2,2,100001,1634,6.69,2007-12-28
3,3,100001,1778,24.79,2008-06-14
4,4,100001,1808,7.44,2008-07-09


In [125]:
gifts['Quater'] = np.where(gifts['date'].dt.month < 4, 'Q1', np.where(gifts['date'].dt.month < 7, 'Q2', np.where(gifts['date'].dt.month < 10, 'Q3','Q4')))
gifts.head()

,giftID,donorID,campaignID,amount,date,Quater
0,0,100001,0,12.39,2004-03-23,Q1
1,1,100001,1577,12.39,2007-10-11,Q4
2,2,100001,1634,6.69,2007-12-28,Q4
3,3,100001,1778,24.79,2008-06-14,Q2
4,4,100001,1808,7.44,2008-07-09,Q3


# DONORS

### Checking data in donors

In [126]:
print(donors.isna().sum().sum())
for column in donors.columns[0:]:
    print(column + " " + str(donors[column].isna().sum()))
donors.head()

34
Unnamed: 0 0
donorID 0
zipcode 2
province 0
region 0
gender 32
language 0
dateOfBirth 0


,Unnamed: 0,donorID,zipcode,province,region,gender,language,dateOfBirth
0,0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967
1,1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952
2,2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986
3,3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952
4,4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986


### Cleaning data in donors

In [127]:
donors_zipcode_na = donors.loc[donors['zipcode'].isna(), :]
donors_zipcode_na.head()
donors['zipcode'] = donors['zipcode'].fillna(0)

In [128]:
donors_gender_na = donors.loc[donors['gender'].isna(), :]
donors_gender_na.head()
donors['gender'] = donors['gender'].fillna('Not Known')

In [129]:
donors.drop(['Unnamed: 0'] , axis = 1, inplace=True)
donors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   donorID      44691 non-null  int64  
 1   zipcode      44691 non-null  float64
 2   province     44691 non-null  object 
 3   region       44691 non-null  object 
 4   gender       44691 non-null  object 
 5   language     44691 non-null  object 
 6   dateOfBirth  44691 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 2.4+ MB


# CAMPAIGNS

### Checking data in campaigns

In [130]:
print(campaigns.isna().sum().sum())
for column in campaigns.columns[0:]:
    print(column + " " + str(campaigns[column].isna().sum()))

campaigns.info()

0
campaignID;date;lettersSent;CostUnit 0
<class 'pandas.core.frame.DataFrame'>
Index: 367 entries, 153;13/12/2004;6 873;0 to 7536;15/12/2019;37 187;0
Data columns (total 1 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   campaignID;date;lettersSent;CostUnit  367 non-null    object
dtypes: object(1)
memory usage: 5.7+ KB


In [131]:
campaigns.head()

,campaignID;date;lettersSent;CostUnit
153;13/12/2004;6 873;0,17 €
154;01/01/2005;7 656;0,30 €
180;18/01/2005;9 933;0,20 €
433;29/01/2005;7 448;0,59 €
476;12/02/2005;6 605;0,51 €


### Cleaning data in campaigns

In [132]:
campaigns = pd.read_csv('./data/campaigns.csv')
campaigns.rename(columns={campaigns.columns[0]: "Cost" }, inplace = True)
campaigns.index.names = ['campaignID;date;lettersSent;CostUnit']
campaigns_reset = campaigns.reset_index()
campaigns_split = campaigns_reset['campaignID;date;lettersSent;CostUnit'].str.split(';', expand=True)
campaigns_split['cost'] = campaigns_reset['Cost']
campaigns_split.rename(columns={0: "campaignID" }, inplace = True)
campaigns_split.rename(columns={1: "date" }, inplace = True)
campaigns_split.rename(columns={2: "lettersSent" }, inplace = True)
campaigns_split.drop([3] , axis = 1, inplace=True)
campaigns_split.head()

,campaignID,date,lettersSent,cost
0,153,13/12/2004,6 873,17 €
1,154,01/01/2005,7 656,30 €
2,180,18/01/2005,9 933,20 €
3,433,29/01/2005,7 448,59 €
4,476,12/02/2005,6 605,51 €


In [133]:
campaigns_split['campaignID'] = campaigns_split['campaignID'].astype(int)
campaigns_split['date'] = pd.to_datetime(campaigns_split['date'])
campaigns_split['lettersSent'] = campaigns_split['lettersSent'].str.replace("\u202f", "").astype(int)
campaigns_split['cost'] = campaigns_split['cost'].str.replace("\xa0€", "").astype(int)
campaigns = campaigns_split
campaigns.head()


,campaignID,date,lettersSent,cost
0,153,2004-12-13,6873,17
1,154,2005-01-01,7656,30
2,180,2005-01-18,9933,20
3,433,2005-01-29,7448,59
4,476,2005-12-02,6605,51


### Creating Features

In [135]:
#Calculating summary for donors till Aug 4th 2018
gifts_donor_campaignID = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['campaignID'].agg({'number_of_campaigns':'count'})
gifts_donor_total_amount = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['amount'].agg({'total_amount':'sum'})
gifts_donor_last_date = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['date'].agg({'last_date':'max'})
gifts_donor_last_date = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['date'].agg({'last_date':'max'})
gifts_donor_quater_contribution = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID', 'Quater'], as_index=False)['date'].agg({'count'}).unstack('Quater')


donors = donors.merge(gifts_donor_campaignID, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_total_amount, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_last_date, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_quater_contribution, left_on = 'donorID', right_on = 'donorID', how = 'left')

donors['days_since_contributed'] = end_date_for_ind_variable - donors['last_date'] 
donors.head() 

C:\tools\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,"(count, Q1)","(count, Q2)","(count, Q3)","(count, Q4)",days_since_contributed
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,3.0,3.0,1.0,5.0,2696 days
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,1.0,NaN,NaN,NaN,1589 days
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,11.0,42.00,2019-05-06,3.0,2.0,5.0,1.0,12 days
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,26.0,716.22,2011-06-03,8.0,4.0,4.0,10.0,2906 days
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,5.0,24.96,2019-04-18,NaN,2.0,NaN,3.0,30 days


In [136]:
#Calculating summary for donors till Aug 4th 2018
gifts_donor_summary = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID'], as_index=False)['amount'].agg(['mean', 'min', 'max'])
gifts_donor_quater_spending = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID', 'Quater'], as_index=False)['amount'].agg({'mean', 'min', 'max', 'sum'}).unstack('Quater')

donors = donors.merge(gifts_donor_summary, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_quater_spending, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors.head()

C:\tools\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,...,"(mean, Q3)","(mean, Q4)","(max, Q1)","(max, Q2)","(max, Q3)","(max, Q4)","(min, Q1)","(min, Q2)","(min, Q3)","(min, Q4)"
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,...,7.44,12.774,20.0,30.0,7.44,24.79,10.00,10.00,7.44,6.69
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,...,NaN,NaN,20.0,NaN,NaN,NaN,20.00,NaN,NaN,NaN
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,11.0,42.00,2019-05-06,...,3.60,4.000,4.0,4.0,5.00,4.00,4.00,4.00,3.00,4.00
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,26.0,716.22,2011-06-03,...,24.79,28.311,30.0,30.0,24.79,49.58,24.79,24.79,24.79,24.79
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,5.0,24.96,2019-04-18,...,NaN,3.320,NaN,10.0,NaN,5.00,NaN,5.00,NaN,2.48


In [137]:
#Calculating age for donors till Aug 4th 2018
donors.loc[:,'age'] = donors['dateOfBirth'].apply(lambda x : end_date_for_ind_variable.year - pd.to_datetime(x).year)
donors.loc[:,'age-group'] = donors['age'] // 10 * 10
donors.head()

,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,...,"(max, Q1)","(max, Q2)","(max, Q3)","(max, Q4)","(min, Q1)","(min, Q2)","(min, Q3)","(min, Q4)",age,age-group
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,...,20.0,30.0,7.44,24.79,10.00,10.00,7.44,6.69,52,50
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,...,20.0,NaN,NaN,NaN,20.00,NaN,NaN,NaN,67,60
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,11.0,42.00,2019-05-06,...,4.0,4.0,5.00,4.00,4.00,4.00,3.00,4.00,33,30
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,26.0,716.22,2011-06-03,...,30.0,30.0,24.79,49.58,24.79,24.79,24.79,24.79,67,60
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,5.0,24.96,2019-04-18,...,NaN,10.0,NaN,5.00,NaN,5.00,NaN,2.48,33,30


In [138]:
gifts.head()

,giftID,donorID,campaignID,amount,date,Quater
0,0,100001,0,12.39,2004-03-23,Q1
1,1,100001,1577,12.39,2007-10-11,Q4
2,2,100001,1634,6.69,2007-12-28,Q4
3,3,100001,1778,24.79,2008-06-14,Q2
4,4,100001,1808,7.44,2008-07-09,Q3


Checking for duplicates to ensure Donor IDs are unique after aggregation

In [139]:
campaigns_campaign_id.nunique()

donorID    25645
dtype: int64

In [140]:
selected_donors = donors.merge(campaigns_campaign_id, left_on='donorID', right_on='donorID', how = 'inner')
selected_donors.head()

,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,...,"(max, Q1)","(max, Q2)","(max, Q3)","(max, Q4)","(min, Q1)","(min, Q2)","(min, Q3)","(min, Q4)",age,age-group
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,...,20.0,30.00,7.44,24.79,10.00,10.00,7.44,6.69,52,50
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,...,20.0,NaN,NaN,NaN,20.00,NaN,NaN,NaN,67,60
2,100008,1780.0,Flemish Brabant,Flanders,M,FR,08/30/1937,3.0,124.37,2011-10-28,...,NaN,49.58,NaN,50.00,NaN,49.58,NaN,24.79,82,80
3,100009,1020.0,Brussels,Brussels,M,FR,01/20/1932,19.0,326.78,2012-03-25,...,20.0,25.00,25.00,20.00,12.39,10.00,20.00,7.44,87,80
4,100010,1020.0,Brussels,Brussels,M,FR,05/30/1983,1.0,40.00,2015-01-13,...,40.0,NaN,NaN,NaN,40.00,NaN,NaN,NaN,36,30


Calculating the number of campaigns each donor has contributed to

In [141]:
# Checking for mutiple donations for campaign campaign_id
gifts1 = gifts[gifts['campaignID'] == campaign_id]
agg_gifts = gifts1.groupby(['donorID'])['date'].agg(['count']).sort_values(by = 'count', ascending =False)


In [142]:
selected_donors = selected_donors.merge(gifts1, left_on='donorID', right_on='donorID', how = 'left')


Setting the target variable

In [143]:
selected_donors['has_contributed'] = np.where((selected_donors['amount'] >= 30) & (selected_donors['date'] > campaign_launched) & (selected_donors['date'] < end_date_tar_variable),1, 0)

Dropping columns from base table that is not required

In [144]:
drop_columns = ['zipcode', 'dateOfBirth', 'last_date', 'giftID', 'campaignID', 'Quater', 'date']
selected_donors = selected_donors.drop(drop_columns, axis = 1)


Renaming all aggregated columns 

In [145]:
selected_donors = selected_donors.rename(columns={('count', 'Q1') : 'Q1_Donations', ('count', 'Q2') : 'Q2_Donations', ('count', 'Q3') : 'Q3_Donations', ('count', 'Q4') : 'Q4_Donations'})


In [146]:
selected_donors = selected_donors.rename(columns={('mean', 'Q1') : 'Q1_Mean', ('mean', 'Q2') : 'Q2_Mean', ('mean', 'Q3') : 'Q3_Mean', ('mean', 'Q4') : 'Q4_Mean'})

In [147]:
selected_donors = selected_donors.rename(columns={('min', 'Q1') : 'Q1_Min', ('min', 'Q2') : 'Q2_Min', ('min', 'Q3') : 'Q3_Min', ('min', 'Q4') : 'Q4_Min'})

In [148]:
selected_donors = selected_donors.rename(columns={('max', 'Q1') : 'Q1_Max', ('max', 'Q2') : 'Q2_Max', ('max', 'Q3') : 'Q3_Max', ('max', 'Q4') : 'Q4_Max'})

In [149]:
selected_donors = selected_donors.rename(columns={('sum', 'Q1') : 'Q1_Amount', ('sum', 'Q2') : 'Q2_Amount', ('sum', 'Q3') : 'Q3_Amount', ('sum', 'Q4') : 'Q4_Amount'})

Converting the dt to days 

In [150]:
selected_donors['days_since_contributed'] = selected_donors['days_since_contributed'].dt.days

In [151]:
selected_donors[selected_donors.has_contributed == 1]

,donorID,province,region,gender,language,number_of_campaigns,total_amount,Q1_Donations,Q2_Donations,Q3_Donations,...,Q3_Max,Q4_Max,Q1_Min,Q2_Min,Q3_Min,Q4_Min,age,age-group,amount,has_contributed
307,100724,Antwerp,Flanders,M,NL,14.0,348.788,3.0,6.0,3.0,...,30.272,27.269,16.500,14.454,13.629,13.629,69,60,40.0,1
396,100964,Antwerp,Flanders,F,NL,5.0,389.400,2.0,1.0,NaN,...,NaN,55.000,55.000,165.000,NaN,39.600,46,40,100.0,1
435,101066,Antwerp,Flanders,M,NL,15.0,136.796,2.0,5.0,4.0,...,22.000,10.912,5.456,5.456,5.456,5.456,77,70,60.0,1
436,101069,Antwerp,Flanders,F,NL,31.0,303.732,4.0,10.0,9.0,...,16.500,11.000,8.184,8.184,8.184,8.184,45,40,80.0,1
604,101426,East Flanders,Flanders,F,NL,2.0,66.000,NaN,NaN,1.0,...,33.000,33.000,NaN,NaN,33.000,33.000,56,50,100.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25330,142982,East Flanders,Flanders,M,NL,1.0,44.000,NaN,NaN,NaN,...,NaN,44.000,NaN,NaN,NaN,44.000,54,50,80.0,1
25420,143214,West Flanders,Flanders,F,NL,1.0,44.000,NaN,NaN,NaN,...,NaN,44.000,NaN,NaN,NaN,44.000,27,20,40.0,1
25426,143231,West Flanders,Flanders,M,NL,2.0,24.629,NaN,NaN,1.0,...,13.629,11.000,NaN,NaN,13.629,11.000,75,70,80.0,1
25466,144371,Limburg,Flanders,M,NL,4.0,181.500,NaN,1.0,2.0,...,49.500,33.000,NaN,55.000,44.000,33.000,70,70,100.0,1


Checking the number of donors for 7244 campaigns

In [152]:
selected_donors.loc[selected_donors.has_contributed == 1, 'has_contributed'].sum()

217

Converting the numerical values to Scaler

In [153]:
scaler = MinMaxScaler()

for column in selected_donors.columns[1:]:
    df = selected_donors[column]

    if selected_donors[column].dtype != object and selected_donors[column].dtype.name != 'category':
        selected_donors[column] = scaler.fit_transform(np.array(selected_donors[[column]]))

In [154]:
encoder = LabelEncoder()
for column in selected_donors.columns[1:]:
    df = selected_donors[column]

    if selected_donors[column].dtype == object or selected_donors[column].dtype.name == 'category':
        selected_donors[column] = encoder.fit_transform(selected_donors[column])

In [155]:
selected_donors.to_csv(os.path.join('./data/tables',r'processed_table_7244.csv'))